In [2]:
# from FlagEmbedding import FlagModel
import numpy as np
import pandas as pd
# import heapq
# import spacy

In [11]:
ts = pd.read_csv('data/tritonsearch_test_results.csv').drop('Class Title', axis=1)
ucs = pd.read_csv('data/ucsd_course_search_test_results.csv').drop('Class Title', axis=1)

test_results = [ts, ucs]

In [15]:
ts.tail(10)

,Query,Class Code,Ranking (position),Relevance
235,nlp,NO RESULTS,1,0
236,nlp,NO RESULTS,2,0
237,nlp,NO RESULTS,3,0
238,nlp,NO RESULTS,4,0
239,nlp,NO RESULTS,5,0
240,javascript,NO RESULTS,1,0
241,javascript,NO RESULTS,2,0
242,javascript,NO RESULTS,3,0
243,javascript,NO RESULTS,4,0
244,javascript,NO RESULTS,5,0


## Mean Reciprocal Rank

In [42]:
# MRR
mrrs = []

for df in test_results:

    # relevant_results = df[df['Relevance'] == 1]

    # # Get the minimum rank for each query, assuming relevance of 1 means a relevant result
    # min_rank_per_query = relevant_results.groupby('Query')['Ranking (position)'].min()

    # # Calculate reciprocal rank
    # reciprocal_rank = 1 / min_rank_per_query

    # # Calculate MRR
    # mrr = reciprocal_rank.mean()

    # -----------------------------------------------------------------------------------------------------------------------------------

    relevant_results_new = df[df['Relevance'] == 1]

    # If a query has no relevant results (i.e., all relevances are 0), we set its minimum rank to infinity for MRR calculation purposes
    # This is because MRR is undefined (or considered as 0) for queries with no relevant results
    df['Reciprocal Rank'] = df.apply(lambda x: 1/x['Ranking (position)'] if x['Relevance'] == 1 else float('inf'), axis=1)
    min_reciprocal_rank_per_query = df.groupby('Query')['Reciprocal Rank'].min()

    # Handle cases where all results are irrelevant by setting their reciprocal rank contribution to 0
    min_reciprocal_rank_per_query = min_reciprocal_rank_per_query.replace(float('inf'), 0)

    # Calculate MRR
    mrr_new = min_reciprocal_rank_per_query.mean()

    mrrs.append(mrr)

mrrs

[0.9895833333333334, 0.9895833333333334]

## NDCG

In [40]:
# Function to calculate DCG
def dcg(scores):
    return np.sum(
        (2**scores - 1) / np.log2(np.arange(2, scores.size + 2))
    )

# Function to calculate NDCG for each query
def ndcg(df):
    df_sorted = df.sort_values(by='Ranking (position)')
    actual_relevances = df_sorted['Relevance'].values
    ideal_relevances = np.sort(df_sorted['Relevance'].values)[::-1]
    
    dcg_actual = dcg(actual_relevances)
    dcg_ideal = dcg(ideal_relevances)
    
    return dcg_actual / dcg_ideal if dcg_ideal > 0 else 0

# Calculate NDCG for each query
ndcg_list = []
for df in test_results:
    ndcg_scores = df.groupby('Query').apply(ndcg, include_groups=False) 
    ndcg_list.append(ndcg_scores.mean())

ndcg_list

[0.7959183673469388, 0.9689886745755573]

In [7]:
# triton_search = pd.read_csv("data/tritonsearch_results.csv")

# queries = triton_search['Query'].unique()
# len(queries)

# data = pd.read_pickle('data/course_catalog_final.pkl')

In [ ]:
# nlp = spacy.load('en_core_web_sm')
# # loading model
# model = FlagModel('BAAI/bge-small-en-v1.5', 
#                 query_instruction_for_retrieval="Represent this sentence for searching relevant passages: ",
#                 use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation;

In [19]:
# def filter(df, springOnly, upper_div, lower_div, graduate, include, exclude):
#     """
#     Optimized filter function for a DataFrame based on level of study and department inclusion/exclusion.

#     Parameters:
#     - df: DataFrame to filter.
#     - upper_div: Boolean, True to include Upper Division levels.
#     - lower_div: Boolean, True to include Lower Division levels.
#     - graduate: Boolean, True to include Graduate levels.
#     - include: String of departments to include (depts separated with commas)
#     - exclude: String of departments to exclude (depts separated with commas)

#     Returns:
#     - Optimized filtered DataFrame based on the specified criteria.
#     """
#     # Create a boolean series for each level condition
#     conditions = pd.Series(False, index=df.index)
#     if upper_div:
#         conditions |= (df['Level'] == 'Upper Division')
#     if lower_div:
#         conditions |= (df['Level'] == 'Lower Division')
#     if graduate:
#         conditions |= (df['Level'] == 'Graduate')
#     if springOnly:
#         conditions &= (df['Spring'] == 'T')
    
#     # Apply level filtering
#     df = df[conditions]

#     # Apply department inclusion and exclusion
#     if include:
#         include_list = include.upper().replace(",", " ").split()
#         print(include_list)
#         df = df[df['Department'].isin(include_list)]
#     if exclude:
#         exclude_list = exclude.upper().replace(",", " ").split()
#         df = df[~df['Department'].isin(exclude_list)]
    
#     # Reset index
#     df = df.reset_index(drop=True)
#     return df
 
# def preprocess_and_embed(text):
#     preprocessed_text = str(nlp(text))
#     return model.encode(preprocessed_text)

# def cosine_similarity(vec1, vec2):
#     """
#     Computes the cosine similarity between two vectors
#     """
#     return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# def search(query, data, k):
#     """
#     Computes the embedding of the query and retrieves the k most similar documents
#     """
#     k = int(k)
#     title_embeddings = data['Title Embeddings']
#     desc_embeddings = data['Description Embeddings']

#     # if the query is a course code, return just the row containing the course code
#     if query.upper() in data['Code'].values:
#         exact_code = data[data['Code'] == query.upper()].iloc[0]
#         return [(exact_code['Code'], exact_code['Title'], exact_code['Description'], exact_code['Prerequisites'], exact_code['URL'], exact_code['Spring'])] 
#     # if the query is a course department, return all courses from that department
#     if query.upper() in data['Department'].values:
#         exact_dept = data[data['Department'] == query.upper()]
#         return exact_dept[['Code', 'Title', 'Description', 'Prerequisites', 'URL', 'Spring']].values.tolist()
    
#     # gets the embedding of the query
#     query_embedding = preprocess_and_embed(query)
    
#     # get the similarities between the query embedding and the title embeddings
#     title_similarities = np.array([cosine_similarity(query_embedding, doc_emb) for doc_emb in title_embeddings])
    
#     # get the similarities between the query embedding and the description embeddings
#     desc_similarities = np.array([cosine_similarity(query_embedding, doc_emb) for doc_emb in desc_embeddings])
    
#     # weight the title and description similarities (weights should add up to 1) and calculate total similarity
#     title_weight = 0.7
#     desc_weight = 0.3
#     similarities = (title_similarities * title_weight) + (desc_similarities * desc_weight)
    
#     # ranks similarities by most similar to query embedding
#     index_similarity_pair_ranked =  heapq.nlargest(k, enumerate(similarities), key=lambda x: x[1])
    
#     ranked_docs = []
#     for ind, sim in index_similarity_pair_ranked:
#         # if cosine similarity < 0.7 and at least 3 courses have been added to search results, stop adding to search results
#         if sim < 0.7 and len(ranked_docs) >= 3:
#             break
#         # return the necessary information
#         ranked_docs.append((data['Code'][ind], data['Title'][ind],  data['Description'][ind]))
    
#     return ranked_docs

In [32]:
# all_results = []

# for q in queries[6:]:
#     results = search(q, data, 5)
#     for r in results:
#         all_results.append([q, r[0], r[1], r[2]])
#     while (len(all_results)%5 != 0):
#         all_results.append([q, 'NO RESULTS', 'NO RESULTS', 'NO RESULTS'])

In [37]:
# ucs_results = pd.DataFrame(all_results)

# ucs_results.columns = ['Query', 'Class Code', 'Class Title', 'Class Description']

# ucs_results.to_csv('ucs_results.csv')